In [1]:
import glob
import pandas as pd
import re
import pandas as pd
from utils.calculate_results_helper import extract_params_from_file_name, refine_output_df

csv_files = glob.glob('data/evaluation_results/*.csv')
dfs = []
empty_n = 0
for file in csv_files:
    if file =='data/evaluation_results/joint_results.csv':
        continue
    tmp_df = pd.read_csv(file)
    if tmp_df.shape[0] == 0:
        print('EMPTY FILE: ', file)
        empty_n += 1
        continue
    print('SOURCE: ', file)
    tmp_df = extract_params_from_file_name(tmp_df)
    dfs.append(tmp_df)

res = pd.concat(dfs)
res = refine_output_df(res)
display(res)
res.to_csv('data/evaluation_results/joint_results.csv')

SOURCE:  data/evaluation_results/zefiro_4bit_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/mistral_noInstr_8bit_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/zefiro_NoQuant_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/zefiro_8bit_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_14B_4bit_FT_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama_13B_4bit_FT_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/mistral_NoQuant_FT_float16.csv
SOURCE:  data/evaluation_results/llama_7B_8bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama_13B_4bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_7B_4bit_FT_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_7B_4bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/mistral_NoQuant_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama_7B_NoQuant_FT_w

,file,f1_score,precision,recall,model_type,instructed,model_configurations,model_size,quantization,fine_tuning,maxNewTokensFactor,nShotsInference,model,bnb_4bit_compute_dtype,r,lora_alpha,lora_dropout,gradient_accumulation_steps,learning_rate
0,data/zefiro/4bit_FT/maxNewTokensFactor8_nShots...,0.400000,0.436886,0.368857,zefiro,instructed,4bit_FT,7,4bit,FT,8,4,zefiro-7b-base-ita_,torch.bfloat16,32.0,64.0,0.01,4.0,0.0002
1,data/zefiro/4bit_FT/maxNewTokensFactor4_nShots...,0.347961,0.420459,0.296788,zefiro,instructed,4bit_FT,7,4bit,FT,4,4,zefiro-7b-base-ita_,torch.bfloat16,64.0,32.0,0.01,8.0,0.0002
2,data/zefiro/4bit_FT/maxNewTokensFactor8_nShots...,0.483046,0.391605,0.630199,zefiro,instructed,4bit_FT,7,4bit,FT,8,0,zefiro-7b-base-ita_,torch.bfloat16,64.0,32.0,0.01,2.0,0.0002
3,data/zefiro/4bit_FT/maxNewTokensFactor8_nShots...,0.414676,0.387327,0.446180,zefiro,instructed,4bit_FT,7,4bit,FT,8,0,zefiro-7b-base-ita_,torch.bfloat16,32.0,64.0,0.01,4.0,0.0002
4,data/zefiro/4bit_FT/maxNewTokensFactor8_nShots...,0.450917,0.394938,0.525387,zefiro,instructed,4bit_FT,7,4bit,FT,8,0,zefiro-7b-base-ita_,torch.bfloat16,64.0,64.0,0.01,8.0,0.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,data/qwen/7B_8bit_base/maxNewTokensFactor4_nSh...,0.497369,0.556719,0.449454,qwen,instructed,7B_8bit_base,7,8bit,base,4,3,BaseModel_Qwen1.5-7B-Chat_8,NaN,NaN,NaN,NaN,NaN,NaN
7,data/qwen/7B_8bit_base/maxNewTokensFactor8_nSh...,0.520493,0.555185,0.489881,qwen,instructed,7B_8bit_base,7,8bit,base,8,1,BaseModel_Qwen1.5-7B-Chat_8,NaN,NaN,NaN,NaN,NaN,NaN
8,data/qwen/7B_8bit_base/maxNewTokensFactor4_nSh...,0.509443,0.575241,0.457153,qwen,instructed,7B_8bit_base,7,8bit,base,4,1,BaseModel_Qwen1.5-7B-Chat_8,NaN,NaN,NaN,NaN,NaN,NaN
9,data/qwen/7B_8bit_base/maxNewTokensFactor4_nSh...,0.508342,0.560437,0.465108,qwen,instructed,7B_8bit_base,7,8bit,base,4,4,BaseModel_Qwen1.5-7B-Chat_8,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
for col in res.columns:
    if col in ['file', 'f1_score', 'recall', 'precision', 'model_configurations', 'model']:
        continue
    print('col: ', col, 'vals: ', res[col].unique())

col:  model_type vals:  ['zefiro' 'mistral' 'qwen' 'llama']
col:  instructed vals:  ['instructed' 'notInstructed']
col:  model_size vals:  ['7' '14' '13']
col:  quantization vals:  ['4bit' '8bit' 'NoQuant']
col:  fine_tuning vals:  ['FT' 'unsure' 'base']
col:  maxNewTokensFactor vals:  ['8' '4' '2']
col:  nShotsInference vals:  ['4' '0' '2' '1' '3']
col:  bnb_4bit_compute_dtype vals:  ['torch.bfloat16' nan]
col:  r vals:  [32. 64. 16. nan]
col:  lora_alpha vals:  [64. 32. nan]
col:  lora_dropout vals:  [0.01  nan 0.05]
col:  gradient_accumulation_steps vals:  [ 4.  8.  2. nan 16.]
col:  learning_rate vals:  ['0.0002' nan '0.0008' '0.002']


In [63]:
res[res['instructed']=='notInstructed']

,file,f1_score,precision,recall,model_type,instructed,model_configurations,model_size,quantization,fine_tuning,maxNewTokensFactor,nShotsInference,model,bnb_4bit_compute_dtype,r,lora_alpha,lora_dropout,gradient_accumulation_steps,learning_rate
0,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.483162,0.502771,0.465024,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
1,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.639212,0.634080,0.644426,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
2,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.559085,0.555629,0.562583,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
3,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.488295,0.495600,0.481203,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
4,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.552100,0.578281,0.528186,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
5,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.635105,0.597381,0.677914,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
6,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.638999,0.583906,0.705571,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
7,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.569430,0.632091,0.518072,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
8,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.649890,0.602338,0.705594,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
9,data/mistral/noInstr_8bit/maxNewTokensFactor8_...,0.580823,0.556810,0.607000,mistral,notInstructed,noInstr_8bit,7,8bit,unsure,8,0,Mistral-7B-v0.1_simplest_prompt,NaN,NaN,NaN,NaN,NaN,NaN
